<h1><center>Mechanisms of Action (MoA) Prediction. </center></h1>


<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:blue; border:0; color:white' role="tab" aria-controls="home"><center>Quick navigation</center></h2>

* [0. Installation of libraries](#0)
* [1. Basic Data Overview](#1)
* [2. Analysis of data](#2)
* [3. EDA](#3)
* [4. Modeling](#4)
* [5. Test sample](#5)
* [6. Comparison of results](#6)
    
    

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from  sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing  import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model 
from sklearn import tree 
from sklearn import ensemble 
from sklearn import metrics 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest, f_classif
import plotly 
import plotly.express as px
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.linear_model import LinearRegression,Ridge,RidgeCV, ElasticNetCV, LassoCV,BayesianRidge
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA


In [2]:
#restart kernel
#from IPython.core.display import HTML
#HTML("<script>Jupyter.notebook.kernel.restart()</script>")

<a id="1"></a>
<h2 style='background:blue; border:0; color:white'><center>1. Basic Data Overview</center><h2>

In [3]:
train_drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv("/kaggle/input/lish-moa/train_targets_nonscored.csv")


In [4]:
print( train_drug.shape, train_features.shape, 
     train_targets_scored.shape, train_targets_nonscored.shape)

(23814, 2) (23814, 876) (23814, 207) (23814, 403)


In [5]:
sample_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')

In [6]:
print(sample_submission.shape, test_features.shape)

(3982, 207) (3982, 876)


<a id="2"></a>
<h2 style='background:blue; border:0; color:white'><center>2. EDA</center><h2>

Work with NaN

In [7]:
a = train_features.isnull().mean()*100
b = a[a>0].sort_values(ascending=False)
b

Series([], dtype: float64)

Categorization

In [8]:
types = train_features.dtypes
cat_features = list(types[(types == 'object')].index)
cat_features

['sig_id', 'cp_type', 'cp_dose']

In [9]:
num_features = list(types[(types == 'int64')].index)
num_features

['cp_time']

In [10]:
from sklearn.preprocessing import LabelBinarizer
transformer = preprocessing.LabelBinarizer()
train_features['cp_type'] = transformer.fit_transform(train_features.cp_type)

train_features['cp_type']

0        1
1        1
2        1
3        1
4        1
        ..
23809    1
23810    1
23811    0
23812    1
23813    1
Name: cp_type, Length: 23814, dtype: int64

In [11]:
cols = ['cp_time', 'cp_dose']
dummie = pd.get_dummies(train_features[cols], drop_first=True)
train_features = pd.concat([train_features, dummie], axis=1)
train_features.drop(cols, axis = 1, inplace=True)

In [12]:
train_features

,sig_id,cp_type,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,...,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99,cp_dose_D2
0,id_000644bb2,1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,...,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176,0
1,id_000779bfc,1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,...,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371,0
2,id_000a6266a,1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,...,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931,0
3,id_0015fd391,1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,...,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154,0
4,id_001626bd3,1,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,...,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,1,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,0.3055,...,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680,1
23810,id_fffb70c0c,1,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,-0.2252,-0.5565,...,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689,1
23811,id_fffc1c3f4,0,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,-0.2541,0.1745,...,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808,1
23812,id_fffcb9e7c,1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,0.0323,0.0463,...,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740,0


<a id="4"></a>
<h2 style='background:blue; border:0; color:white'><center>4. Modeling</center><h2>

In [35]:
X = train_features.drop(['sig_id'], axis = 1)
y = train_targets_scored.drop(['sig_id'], axis = 1)

In [14]:
cols = list(y.columns)

In [15]:
from imblearn.over_sampling import RandomOverSampler

In [16]:
y_x = y['5-alpha_reductase_inhibitor']
ros = RandomOverSampler(random_state=2)
X_s, y_s = ros.fit_resample(X, y_x)
X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, random_state = 42, test_size = 0.33)
pipeline = Pipeline([('scaler', MinMaxScaler()), 
                    ('pca',PCA(n_components=50)), 
                    ('dt', DecisionTreeClassifier(random_state=42))])
pipeline.fit(X_train, y_train)  
y_pred_train = pipeline.predict(X_train)
y_pred_test = pipeline.predict(X_test)
print('Train df:',metrics.classification_report(y_train, y_pred_train))
print('Test df:',metrics.classification_report(y_test, y_pred_test))

Train df:               precision    recall  f1-score   support

           0       0.90      1.00      0.95     15909
           1       1.00      0.88      0.94     15978

    accuracy                           0.94     31887
   macro avg       0.95      0.94      0.94     31887
weighted avg       0.95      0.94      0.94     31887

Test df:               precision    recall  f1-score   support

           0       0.89      1.00      0.94      7888
           1       1.00      0.88      0.94      7819

    accuracy                           0.94     15707
   macro avg       0.95      0.94      0.94     15707
weighted avg       0.95      0.94      0.94     15707



In [17]:
train = list()
test = list()

In [18]:
for col in cols:
    y_x = y[col]
    ros = RandomOverSampler(random_state=2)
    X_s, y_s = ros.fit_resample(X, y_x)
    X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, random_state = 42, test_size = 0.33)
    pipeline = Pipeline([('scaler', MinMaxScaler()), 
                         ('pca',PCA(n_components=50)), 
                         ('dt', DecisionTreeClassifier(random_state=42))])
    pipeline.fit(X_train, y_train)  
    y_pred_train = pipeline.predict(X_train)
    y_pred_test = pipeline.predict(X_test)
    f1_score_train = metrics.f1_score(y_train, y_pred_train)
    f1_score_test = metrics.f1_score(y_test, y_pred_test)
    train.append(f1_score_train)
    test.append(f1_score_test)

In [19]:
print(len(cols), len(train), len(test))

206 206 206


In [20]:
df = pd.DataFrame({'column':cols, 'train_score':train, 'test_score':test})
df

,column,train_score,test_score
0,5-alpha_reductase_inhibitor,0.865038,0.864629
1,11-beta-hsd1_inhibitor,0.970976,0.969258
2,acat_inhibitor,0.906021,0.910977
3,acetylcholine_receptor_agonist,0.863579,0.851727
4,acetylcholine_receptor_antagonist,0.900470,0.891022
...,...,...,...
201,ubiquitin_specific_protease_inhibitor,0.999937,0.999872
202,vegfr_inhibitor,0.922560,0.914889
203,vitamin_b,0.939010,0.936257
204,vitamin_d_receptor_agonist,0.986707,0.988076


In [21]:
import plotly.express as px
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['column'], y=df['train_score'],
                    mode='lines',
                    name='Train Score',
                    line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df['column'], y=df['test_score'],
                    mode='lines',
                    name='Test Score',
                    line=dict(color='red')))
fig.show()

<a id="5"></a>
<h2 style='background:blue; border:0; color:white'><center>5. Test sample</center><h2>

In [36]:
X_test = test_features.drop(['sig_id'], axis = 1)
X_test['cp_type'] = transformer.fit_transform(X_test.cp_type)
col_cat = ['cp_time', 'cp_dose']
dummie = pd.get_dummies(X_test[col_cat], drop_first=True)
X_test = pd.concat([X_test, dummie], axis=1)
X_test.drop(col_cat, axis = 1, inplace=True)

In [38]:
for col in cols:
    y_x = y[col]
    ros = RandomOverSampler(random_state=2)
    X_s, y_s = ros.fit_resample(X, y_x)
    pipeline = Pipeline([('scaler', MinMaxScaler()), 
                         ('pca',PCA(n_components=50)), 
                         ('dt', DecisionTreeClassifier(random_state=42))])
    pipeline.fit(X_s, y_s)
    y_pred_test = pipeline.predict(X_test)
    sample_submission[col]=y_pred_test

In [40]:
sample_submission[sample_submission['adenosine_receptor_agonist']==1]

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
71,id_04fa00d59,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
198,id_0d89f05f6,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
204,id_0e04748b4,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
438,id_1cffe68af,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
472,id_1f7e1cef4,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1216,id_4ec6fe4a3,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1221,id_4f334570a,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1777,id_722d4d021,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1995,id_8095ab548,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2057,id_84634f39d,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


<a id="6"></a>
<h2 style='background:blue; border:0; color:white'><center>6. Comparison of results</center><h2>

In [47]:
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')

In [48]:
train_df = train_features.merge(right=train_targets_scored, on = 'sig_id', how = 'outer')
train_df.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0,0,0,0,0,0,0,0,0,0


In [46]:
test_df = test_features.merge(right=sample_submission, on = 'sig_id', how = 'outer')
test_df.head()


,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0,0,0,0,0,0,0,0,0,0
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,0,0,0,0,0,0,0,0,0,0
2,id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,0,0,0,0,0,0,0,0,0,0
3,id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0,0,0,0,0,0,0,0,0,0
4,id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0,0,0,0,0,0,0,0,0,0


In [50]:
import random

In [91]:
random_col = random.sample(cols, 50)
random_train = train_df[random_col]
random_test = test_df[random_col]

a = random_train[random_col].mean().reset_index()
b = random_test[random_col].mean().reset_index()


In [92]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(x=a['index'], y=a[0], name='Train Score'))

fig.add_trace(go.Scatter(x=b['index'], y=b[0], mode='lines', name='Test Score', line=dict(color='red')))

fig.show()
